In [5]:
import re

legalmora = []
for i in range(1,7):
    print((i-1)*4  + 1)
    curmora = (i-1)*4  + 1
    legalmora.append(curmora)
    curmora = curmora + 2
    legalmora.append(curmora)
    if(i<6):
        curmora = curmora + 1
        legalmora.append(curmora)

print(legalmora)

1
5
9
13
17
21
[1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 21, 23]


In [4]:
import re

full_lines = {}
meter_lines = {}
caesuras = {}
caesmasc3 = {}
caesmasc4 = {}
caesfem3 = {}
caesfem4 = {}
bucdiaer = {}
hermannbroken = {}
is_enclitic = {}

is_enclitic['γε'] = 1
is_enclitic['γέ'] = 1
is_enclitic["γ’"] = 1
is_enclitic['κε'] = 1
is_enclitic['κεν'] = 1
is_enclitic['ἑ'] = 1
is_enclitic['κέ'] = 1
is_enclitic["κ’"] = 1
is_enclitic['με'] = 1
is_enclitic['μέ'] = 1
is_enclitic["μ’"] = 1
is_enclitic['μευ'] = 1
is_enclitic['μεύ'] = 1
is_enclitic['μιν'] = 1
is_enclitic['μοι'] = 1
is_enclitic['οἱ'] = 1
is_enclitic["περ"] = 1
is_enclitic["πέρ"] = 1
is_enclitic["ποθ’"] = 1
is_enclitic["ποτ’"] = 1
is_enclitic["ποτε"] = 1
is_enclitic["που"] = 1
is_enclitic["πού"] = 1

is_enclitic["πω"] = 1
is_enclitic["πώ"] = 1
is_enclitic['ῥα'] = 1
is_enclitic['σε'] = 1
is_enclitic['σέ'] = 1
is_enclitic['σευ'] = 1
is_enclitic['σεύ'] = 1
is_enclitic["σ’"] = 1
is_enclitic["σφιν"] = 1
is_enclitic["σφι"] = 1
is_enclitic['τε'] = 1
is_enclitic['τέ'] = 1
is_enclitic["τ’"] = 1
is_enclitic["τι"] = 1
is_enclitic["τινας"] = 1
is_enclitic["τινες"] = 1
is_enclitic["τις"] = 1
is_enclitic["τοι"] = 1
is_enclitic["τοί"] = 1

nocaesurabefore = {}
nocaesurabefore['ἄν'] = 1
nocaesurabefore['ἂν'] = 1
nocaesurabefore['δέ'] = 1
nocaesurabefore['δὲ'] = 1
nocaesurabefore['δ’'] = 1
nocaesurabefore['μέν'] = 1
nocaesurabefore['μὲν'] = 1
nocaesurabefore['γάρ'] = 1
nocaesurabefore['γὰρ'] = 1


nocaesuraafter = {}

nocaesuraafter['ἀλλ’'] = 1
nocaesuraafter['ἀλλὰ'] = 1
nocaesuraafter['ἀλλά'] = 1
nocaesuraafter['καί'] = 1
nocaesuraafter['καὶ'] = 1
wordstarts = {}
for i in range(1,25):
    wordstarts[i] = 0
    
def getlines(fname):
    f = open(fname)
    nofollowingcaes = 0
    for l in f:
        if(re.search('^#',l)):
            continue
        l = re.sub('\n$','',l)
        args = l.split()
        curcit = args[0]
        curform = args[1]
        if(curcit in full_lines):
            full_lines[curcit] = full_lines[curcit] + ' ' + curform
        else:
            full_lines[curcit] = curform
        if(curcit in meter_lines):
            meter_lines[curcit] = meter_lines[curcit] + ' ' + args[4]
        else:
            meter_lines[curcit] = args[4]
        #print(args[3])
        startpart = int(args[2])
        wordstarts[startpart] = wordstarts[startpart] + 1
        endpart = int(args[3])
        wordlen = 0
        workw,wordlen = re.subn('short','',l)
        workw,n = re.subn('long','',l)
        wordlen = wordlen + n * 2
        rightlen = endpart - startpart + 1
        if(wordlen and not rightlen == wordlen):
            
            print(wordlen,l)
        
        if(endpart>24):
            print('start',l)
        if(nofollowingcaes):
            nofollowingcaes = 0
            continue
        if(curform in nocaesuraafter):
            nofollowingcaes = 1
        if(curform in is_enclitic or curform in nocaesurabefore):
            #print('skipping',curcit,curform)
            continue
        if(startpart == 11):
            caesmasc3[curcit]  = l
            continue
        if(startpart == 12):
            caesfem3[curcit] = l
            continue
        if(startpart == 15):
            caesmasc4[curcit]  = l
            continue
        if(startpart == 16):
            caesfem4[curcit]  = l
            continue
        if(startpart == 17):
            bucdiaer[curcit] = l
            continue
    f.close()
    
getlines('tlg0012.tlg001.meter.tsv')
getlines('tlg0012.tlg002.meter.tsv')

def get_meter_stats(s):
    l = meter_lines[s]
    consonants = '[βγδζθκλμνπξρστςφψχ]'
    doublecons = 'short-(.[βγδζθκλμνπξρστςφψχ]\\s*[βγδζθκλμνπξρστςφψχ]+)'

    noelision = '([εαο]\\s(long|short)-[ἐἀὀἔἄὄἠὠἤὤ])'
    #print(l)
    while(re.search(doublecons,l)):
        m = re.search(doublecons,l)
        print('short',s,m[1])
        l = re.sub(doublecons,'',l,1)
        
    while(re.search(noelision,l)):
        m = re.search(noelision,l)
        print(s,'noelis',m[1],full_lines[s])
        l = re.sub(noelision,'',l,1)
        
    synezisis = '(long-ε[ωῶῷῳ])'
    while(re.search(synezisis,l)):
        m = re.search(synezisis,l)
        print(s,'synezisis',m[1],full_lines[s])
        l = re.sub(synezisis,'',l,1)
        
    
    
for foo in sorted(wordstarts,key=wordstarts.get,reverse=True):
    footnum = 1 + (foo-1)*.25
    print(foo,footnum,wordstarts[foo])
caes3 = 0
caes4 = 0

print('caesmasc3',len(caesmasc3))
print('caesfem3',len(caesfem3))
print('caesmasc4',len(caesmasc4))
print('caesfem4',len(caesfem4))

for foo in full_lines:
    get_meter_stats(foo)
    if(foo in caesmasc3 and foo in caesfem3):
        print('m+f',foo,full_lines[foo])
    if(foo in caesmasc3 or foo in caesfem3):
        caes3 = caes3 + 1
        continue
    if(foo in caesmasc4):
        caes4 = caes4 + 1
        continue

    print()
    print(foo)
    if(foo in caesmasc4):
        print('masc4',caesmasc4[foo])
    if(foo in caesfem4):
        print('fem4',caesfem4[foo])
    if(foo in bucdiaer):
        print('bucdiaer',bucdiaer[foo])
    print(full_lines[foo])
    print(meter_lines[foo])

print('done',len(full_lines),'caes3',caes3,caes3/len(full_lines),'caes4',caes4,caes4/len(full_lines))

1 1.0 27791
7 2.5 18141
17 5.0 17103
12 3.75 17005
5 2.0 16544
15 4.5 14191
11 3.5 14134
20 5.75 13849
3 1.5 12555
4 1.75 8697
21 6.0 8505
19 5.5 8433
13 4.0 7092
9 3.0 6913
8 2.75 4579
16 4.75 1318
23 6.5 1026
2 1.25 10
6 2.25 5
10 3.25 2
14 4.25 1
18 5.25 0
22 6.25 0
24 6.75 0
caesmasc3 11763
caesfem3 16160
caesmasc4 12869
caesfem4 216
tlg001:1.1 synezisis long-εω μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος
tlg001:1.7 noelis ε short-ἄ Ἀτρεΐδης τε ἄναξ ἀνδρῶν καὶ δῖος Ἀχιλλεύς
m+f tlg001:1.33 ὣς ἔφατ’ ἔδεισεν δ’ ὃ γέρων καὶ ἐπείθετο μύθῳ
m+f tlg001:1.68 ἤτοι ὅ γ’ ὣς εἰπὼν κατ’ ἄρ’ ἕζετο τοῖσι δ’ ἀνέστη
tlg001:1.75 noelis ο short-ἄ μῆνιν Ἀπόλλωνος ἑκατηβελέταο ἄνακτος
m+f tlg001:1.101 ἤτοι ὅ γ’ ὣς εἰπὼν κατ’ ἄρ’ ἕζετο τοῖσι δ’ ἀνέστη
tlg001:1.157 noelis ε long-ἠ οὔρεά τε σκιόεντα θάλασσά τε ἠχήεσσα
tlg001:1.172 noelis α short-ἄ τὸν δ’ ἠμείβετ’ ἔπειτα ἄναξ ἀνδρῶν Ἀγαμέμνων
m+f tlg001:1.227 οὔτε λόχον δ’ ἰέναι σὺν ἀριστήεσσιν Ἀχαιῶν
m+f tlg001:1.277 μήτε σὺ Πηλείδη θελ’ ἐριζέμεναι βασιλῆϊ
m+f tlg0

m+f tlg001:16.444 ἄλλο δέ τοι ἐρέω σὺ δ’ ἐνὶ φρεσὶ βάλλεο σῇσιν
tlg001:16.456 noelis ε short-ἔ ἔνθά ἑ ταρχύσουσι κασίγνητοί τε ἔται τε
tlg001:16.512 noelis ο short-ἀ τείχεος ὑψηλοῖο ἀρὴν ἑτάροισιν ἀμύνων
tlg001:16.554 synezisis long-εω ὦρσε Μενοιτιάδεω Πατροκλῆος λάσιον κῆρ
m+f tlg001:16.579 ἐν κόρυθι βριαρῇ ὃ δ’ ἄρα πρηνὴς ἐπὶ νεκρῷ
tlg001:16.653 synezisis long-εω ὄφρ’ ἠῢς θεράπων Πηληϊάδεω Ἀχιλῆος
tlg001:16.674 noelis ε short-ἔ ἔνθά ἑ ταρχύσουσι κασίγνητοί τε ἔται τε
m+f tlg001:16.742 αὐτοῦ πρόσθε ποδῶν ὃ δ’ ἄρ’ ἀρνευτῆρι ἐοικὼς
tlg001:16.769 noelis ε long-ἀ ἠχῇ θεσπεσίῃ πάταγος δέ τε ἀγνυμενάων
tlg001:16.781 noelis α short-ἔ ἐκ μὲν Κεβριόνην βελέων ἥρωα ἔρυσσαν
tlg001:16.804 noelis α short-ἄ λῦσε δέ οἱ θώρηκα ἄναξ Διὸς υἱὸς Ἀπόλλων
tlg001:16.848 noelis ο short-ἐ πάντές κ’αὐτόθ’ ὄλοντο ἐμῷ ὑπὸ δουρὶ δαμέντες
m+f tlg001:16.851 ἄλλο δέ τοι ἐρέω σὺ δ’ ἐνὶ φρεσὶ βάλλεο σῇσιν
short tlg001:16.857 ἀνδρ
tlg001:17.16 noelis ε short-ἔ τώ με ἔα κλέος ἐσθλὸν ἐνὶ Τρώεσσιν ἀρέσθαι
m+f tlg001:17.54

tlg002:9.83 noelis α short-ἀ πόντον ἐπ’ ἰχθυόεντα ἀτὰρ δεκάτῃ ἐπέβημεν
tlg002:9.99 noelis α short-ἐ νηυσὶ δ’ ἐνὶ γλαφυρῇσιν ὑπὸ ζυγὰ δῆσα ἐρύσσας
tlg002:9.159 noelis α long-ἐ νῆες μέν μοι ἕποντο δυώδεκα ἐς δὲ ἑκάστην
tlg002:9.194 noelis α short-ἔ αὐτοῦ πὰρ&ampnnbspνηΐ τε μένειν καὶ νῆα ἔρυσθαι
tlg002:9.210 noelis α short-ἀ χεῦ’ ὀδμὴ δ’ ἡδεῖα ἀπὸ κρητῆρος ὀδώδει
short tlg002:9.242 υκλ
tlg002:9.263 synezisis long-εω λαοὶ δ’ Ἀτρεΐδεω Ἀγαμέμνονος εὐχόμεθ’ εἶναι
tlg002:9.295 noelis α long-ἔ σχέτλια ἔργ’ ὁρόωντες ἀμηχανίη δ’ ἔχε θυμόν
m+f tlg002:9.315 πολλῇ δὲ ῥοίζῳ πρὸς ὄρος τρέπε πίονα μῆλα
m+f tlg002:9.374 ψωμοί τ’ ἀνδρόμεοι ὁ δ’ ἐρεύγετο οἰνοβαρείων
tlg002:9.376 noelis ο short-ἔ εἵως θερμαίνοιτο ἔπεσσι δὲ πάντας ἑταίρους
tlg002:9.438 noelis ο long-ἄ καὶ τότ’ ἔπειτα νομόνδ’ ἐξέσσυτο ἄρσενα μῆλα
tlg002:9.440 noelis ο short-ἄ οὔθατα γὰρ σφαραγεῦντο ἄναξ δ’ ὀδύνῃσι κακῇσι
m+f tlg002:9.453 ὀφθαλμὸν ποθέεις τὸν ἀνὴρ κακὸς ἐξαλάωσε
m+f tlg002:9.489 ἐμβαλέειν κώπῃς ἵν’ ὑπὲκ κακότητα φύγοιμεν
m+f

m+f tlg002:21.374 τῶ κε τάχα στυγερῶς τιν’ ἐγὼ πέμψαιμι νέεσθαι
tlg002:21.395 noelis ο short-ἄ μὴ κέρα ἶπες ἔδοιεν ἀποιχομένοιο ἄνακτος
tlg002:21.415 synezisis long-εω ὅττι ῥά οἱ τέρας ἧκε Κρόνου πάϊς ἀγκυλομήτεω·
tlg002:21.421 synezisis long-εω ἄντα τιτυσκόμενος πελέκεων δ’ οὐκ ἤμβροτε πάντων
tlg002:22.49 noelis ε long-ἔ Ἀντίνοος· οὗτος γὰρ ἐπίηλεν τάδε ἔργα
m+f tlg002:22.81 σμερδαλέα ἰάχων· ὁ δ’ ἁμαρτῇ δῖος Ὀδυσσεὺς
tlg002:22.119 noelis α short-ἄ αὐτὰρ ἐπεὶ λίπον ἰοὶ ὀϊστεύοντα ἄνακτα
m+f tlg002:22.121 ἔκλιν’ ἑστάμεναι πρὸς ἐνώπια παμφανόωντα
tlg002:22.131 synezisis long-εω τοῖς δ’ Ἀγέλεως μετέειπεν ἔπος πάντεσσι πιφαύσκων·
tlg002:22.149 noelis ο long-ἔ μακρὰ τινάσσοντας· μέγα δ’ αὐτῷ φαίνετο ἔργον
m+f tlg002:22.168 ἦε σοὶ ἐνθάδ’ ἄγω ἵν’ ὑπερβασίας ἀποτίσῃ
tlg002:22.185 synezisis long-εω Λαέρτεω ἥρωος ὃ κουρίζων φορέεσκε·
tlg002:22.196 noelis ε short-ἔ εὐνῇ ἔνι μαλακῇ καταλέγμενος ὥς σε ἔοικεν·
tlg002:22.233 noelis ε long-ἔ ἀλλ’ ἄγε δεῦρο πέπον παρ’ ἔμ’ ἵστασο καὶ ἴδε ἔργον
m+f tlg00

In [6]:
import re

f = open('tlg0012.tlg001.meter.tsv')
outf = open('meter.tmp','w')
prevcit = ''
for l in f:
    if(re.search('^#',l)):
        print(l,end='',file=outf)
        continue
    args = l.split()
    curform = args[1]
    startplace = int(args[2])
    endplace = int(args[3])
    meterform = args[4]
    
    curcit = args[0]
    if(not curcit == prevcit):
        diffval = startplace - 1
    startplace = startplace - diffval
    endplace = endplace - diffval
    #if(diffval):
        
    #    print('a',l,end='')
    #    print('b',curcit,curform,startplace,endplace,meterform)
    prevcit = curcit
    print(curcit,curform,startplace,endplace,meterform,sep='\t',file=outf) 

outf.close()
f.close()

In [15]:
len(caesfem4)
caesfem4

{'tlg001:1.20': 'tlg001:1.20\tἄποινα\t16\t19\tδ’-short-ἄπ-long-οιν-short-α',
 'tlg001:1.137': 'tlg001:1.137\tκεν\t16\t16\tκ-short-εν',
 'tlg001:1.168': 'tlg001:1.168\tκάμω\t16\t18\tκ-short-άμ-long-ω',
 'tlg001:1.174': 'tlg001:1.174\tἔμοιγε\t16\t19\tshort-ἔμ-long-οιγ-short-ε',
 'tlg001:1.235': 'tlg001:1.235\tὄρεσσι\t16\t19\tshort-ὄρ-long-εσσ-short-ι',
 'tlg001:1.324': 'tlg001:1.324\tκεν\t16\t16\tκ-short-εν',
 'tlg001:1.350': 'tlg001:1.350\tἀπείρονα\t16\t20\tshort-ἀπ-long-είρ-short-ον-short-α',
 'tlg001:1.576': 'tlg001:1.576\tχερείονα\t16\t20\tχ-short-ερ-long-εί-short-ον-short-α',
 'tlg001:2.239': 'tlg001:2.239\tἀμείνονα\t16\t20\tshort-ἀμ-long-είν-short-ον-short-α',
 'tlg001:2.246': 'tlg001:2.246\tἐὼν\t16\t18\tshort-ἐ-long-ὼν',
 'tlg001:2.250': 'tlg001:2.250\tἔχων\t16\t18\tshort-ἔχ-long-ων',
 'tlg001:2.475': 'tlg001:2.475\tνομῷ\t16\t18\tν-short-ομ-long-ῷ',
 'tlg001:2.479': 'tlg001:2.479\tΠοσειδάωνι\t16\t24\tΠ-short-οσ-long-ειδ-long-ά-long-ων-long-ι',
 'tlg001:2.529': 'tlg001:2.529\tἔην\t

In [21]:
len(full_lines)

27791